# Yahoo-Fin Testing

In [1]:
import requests
import pandas as pd
import ftplib
import io
import re
from requests_html import HTMLSession
from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem

## Configuración inicial

In [2]:
# Rotador de user-agent
software_names = [SoftwareName.CHROME.value, SoftwareName.FIREFOX.value, SoftwareName.EDGE.value]
operating_systems = [OperatingSystem.LINUX.value, OperatingSystem.WINDOWS.value, OperatingSystem.MAC.value]
user_agent_rotator = UserAgent(software_names=software_names, operating_systems=operating_systems, limit=100)

In [3]:
# Punto de API
base_url = "https://query1.finance.yahoo.com/v8/finance/chart/"

In [4]:
# Auxiliar para construir la llamada de la API
def build_url(ticker, start_date=None, end_date=None, interval="1d"):
    if end_date is None:
        end_seconds = int(pd.Timestamp("now").timestamp())

    else:
        end_seconds = int(pd.Timestamp(end_date).timestamp())

    if start_date is None:
        start_seconds = 7223400

    else:
        start_seconds = int(pd.Timestamp(start_date).timestamp())

    site = base_url + ticker

    headers = {'User-Agent': user_agent_rotator.get_random_user_agent()}

    if interval == '1m':
        params = {"period1": start_seconds, "period2": end_seconds,
                  "interval": interval.lower(), "events": "div,splits",
                  'range': '1d'}
    else:
        params = {"period1": start_seconds, "period2": end_seconds,
                  "interval": interval.lower(), "events": "div,splits"}

    return site, headers, params

## Prueba

In [5]:
ticker = 'KOFUBL.MX'
start_date = None
end_date = None
interval = '1m'

In [6]:
site, headers, params = build_url(ticker, start_date, end_date, interval)

In [10]:
site

'https://query1.finance.yahoo.com/v8/finance/chart/KOFUBL.MX'

In [9]:
headers

{'User-Agent': 'Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1204.0 Safari/537.1'}

In [11]:
params

{'period1': 7223400,
 'period2': 1580405471,
 'interval': '1m',
 'events': 'div,splits',
 'range': '1d'}

In [12]:
resp = requests.get(site, headers=headers, params=params)

In [14]:
data = resp.json()

In [15]:
data

{'chart': {'result': [{'meta': {'currency': 'MXN',
     'symbol': 'KOFUBL.MX',
     'exchangeName': 'MEX',
     'instrumentType': 'EQUITY',
     'firstTradeDate': 1554989400,
     'regularMarketTime': 1580417997,
     'gmtoffset': -21600,
     'timezone': 'CST',
     'exchangeTimezoneName': 'America/Mexico_City',
     'regularMarketPrice': 115.78,
     'chartPreviousClose': 124.44,
     'previousClose': 115.76,
     'scale': 3,
     'priceHint': 2,
     'currentTradingPeriod': {'pre': {'timezone': 'CST',
       'start': 1580394600,
       'end': 1580394600,
       'gmtoffset': -21600},
      'regular': {'timezone': 'CST',
       'start': 1580394600,
       'end': 1580418000,
       'gmtoffset': -21600},
      'post': {'timezone': 'CST',
       'start': 1580418000,
       'end': 1580418000,
       'gmtoffset': -21600}},
     'tradingPeriods': [[{'timezone': 'CDT',
        'start': 1554989400,
        'end': 1555012800,
        'gmtoffset': -18000}],
      [{'timezone': 'CDT',
        's

In [16]:
pd.DataFrame(data["chart"]["result"][0]["indicators"]["quote"][0])

,open,close,low,volume,high
0,115.760002,115.790001,115.739998,0.0,115.790001
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
385,115.839996,115.839996,115.839996,1100.0,115.839996
386,115.839996,115.830002,115.830002,2661.0,115.860001
387,115.800003,115.779999,115.769997,980.0,115.800003
388,115.790001,115.550003,115.550003,11546.0,115.800003
